In [4]:
%%capture
! pip install mlxtend
! pip install openpyxl

In [5]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Load the data from the UCI Machine Learning Repository
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
online_retail_data = pd.read_excel(url)

In [6]:
# Data preprocessing
online_retail_data["Description"] = online_retail_data["Description"].str.strip()
online_retail_data = online_retail_data.dropna(axis=0, subset=["InvoiceNo"])
online_retail_data["InvoiceNo"] = online_retail_data["InvoiceNo"].astype("str")
online_retail_data = online_retail_data[
    ~online_retail_data["InvoiceNo"].str.contains("C")
]  # 취소

In [7]:
online_retail_data.sample(20)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
37820,539484,22180,RETROSPOT LAMP,1,2010-12-19 15:44:00,9.95,15640.0,United Kingdom
329213,565841,21933,PINK VINTAGE PAISLEY PICNIC BAG,4,2011-09-07 12:04:00,2.46,NaN,United Kingdom
37735,539475,22556,PLASTERS IN TIN CIRCUS PARADE,1,2010-12-19 14:41:00,1.65,16686.0,United Kingdom
201299,554278,22607,WOODEN ROUNDERS GARDEN SET,2,2011-05-23 13:25:00,9.95,13831.0,United Kingdom
73083,542284,22606,WOODEN SKITTLES GARDEN SET,4,2011-01-27 10:56:00,12.75,17311.0,United Kingdom
151637,549563,22996,TRAVEL CARD WALLET VINTAGE TICKET,3,2011-04-10 14:02:00,0.42,14030.0,United Kingdom
332863,566124,22470,HEART OF WICKER LARGE,6,2011-09-09 11:15:00,2.95,16515.0,United Kingdom
459440,575875,22446,PIN CUSHION BABUSHKA PINK,1,2011-11-11 13:06:00,2.46,NaN,United Kingdom
347241,567294,23394,POSTE FRANCE CUSHION COVER,4,2011-09-19 12:06:00,3.75,13774.0,United Kingdom
241520,558232,20679,EDWARDIAN PARASOL RED,1,2011-06-27 14:30:00,5.95,17841.0,United Kingdom


In [8]:
# Convert transaction data into a basket format
basket = (
    online_retail_data[online_retail_data["Country"] == "United Kingdom"]
    .groupby(["InvoiceNo", "Description"])["Quantity"]
    .sum()
    .unstack()
    .reset_index()
    .fillna(0)
    .set_index("InvoiceNo")
)


# Convert the quantities into 0/1 (0: not in the basket, 1: in the basket)
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1


basket_sets = basket.applymap(encode_units)

# Use the Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

/opt/anaconda3/envs/tf-env/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [9]:
basket.shape

(18667, 4175)

In [10]:
basket.head()

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.046928,0.049821,0.030160,0.642694,12.900183,1.0,0.027822,2.659288,0.967903,0.452936,0.623960,0.624035
1,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.049821,0.046928,0.030160,0.605376,12.900183,1.0,0.027822,2.415142,0.970850,0.452936,0.585946,0.624035
2,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.050035,0.037660,0.030910,0.617773,16.403939,1.0,0.029026,2.517719,0.988498,0.544340,0.602815,0.719271
3,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037660,0.050035,0.030910,0.820768,16.403939,1.0,0.029026,5.300203,0.975787,0.544340,0.811328,0.719271
4,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.050035,0.051267,0.037553,0.750535,14.639752,1.0,0.034988,3.803076,0.980765,0.589076,0.737055,0.741516
5,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.051267,0.050035,0.037553,0.732497,14.639752,1.0,0.034988,3.551237,0.982039,0.589076,0.718408,0.741516
6,(JUMBO BAG BAROQUE BLACK WHITE),(JUMBO BAG RED RETROSPOT),0.048749,0.103820,0.030535,0.626374,6.033290,1.0,0.025474,2.398601,0.877006,0.250219,0.583090,0.460246
7,(JUMBO BAG RED RETROSPOT),(JUMBO BAG BAROQUE BLACK WHITE),0.103820,0.048749,0.030535,0.294118,6.033290,1.0,0.025474,1.347605,0.930898,0.250219,0.257943,0.460246
8,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.062088,0.103820,0.042053,0.677308,6.523895,1.0,0.035607,2.777201,0.902769,0.339533,0.639925,0.541182
9,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.103820,0.062088,0.042053,0.405057,6.523895,1.0,0.035607,1.576473,0.944807,0.339533,0.365673,0.541182


In [12]:
# Filter rules by a minimum lift and confidence
filtered_rules = rules[(rules["lift"] >= 3.0) & (rules["confidence"] >= 0.5)]

In [13]:
# Print the association rules
print(filtered_rules[["antecedents", "consequents", "support", "confidence", "lift"]])

                            antecedents                        consequents  \
0          (ALARM CLOCK BAKELIKE GREEN)         (ALARM CLOCK BAKELIKE RED)   
1            (ALARM CLOCK BAKELIKE RED)       (ALARM CLOCK BAKELIKE GREEN)   
2     (GREEN REGENCY TEACUP AND SAUCER)   (PINK REGENCY TEACUP AND SAUCER)   
3      (PINK REGENCY TEACUP AND SAUCER)  (GREEN REGENCY TEACUP AND SAUCER)   
4     (GREEN REGENCY TEACUP AND SAUCER)  (ROSES REGENCY TEACUP AND SAUCER)   
5     (ROSES REGENCY TEACUP AND SAUCER)  (GREEN REGENCY TEACUP AND SAUCER)   
6      (JUMBO  BAG BAROQUE BLACK WHITE)          (JUMBO BAG RED RETROSPOT)   
8             (JUMBO BAG PINK POLKADOT)          (JUMBO BAG RED RETROSPOT)   
11  (JUMBO SHOPPER VINTAGE RED PAISLEY)          (JUMBO BAG RED RETROSPOT)   
13             (JUMBO STORAGE BAG SUKI)          (JUMBO BAG RED RETROSPOT)   

     support  confidence       lift  
0   0.030160    0.642694  12.900183  
1   0.030160    0.605376  12.900183  
2   0.030910    0.617773  1